1) Description

I analyse two tables from a UBC Minecraft research server: players.csv (196 observations, 7 variables) contains one row per participant, while sessions.csv (1,535 observations, 5 variables) contains one row per gameplay session. players.csv records who each player is and whether they opted into the newsletter; sessions.csv records when each session started and ended. At a glance, a few practical issues are worth noting: some time stamps may not parse cleanly; a number of players appear in players.csv but never show up in the sessions table; some fields can be missing or inconsistently filled; time zones and daylight-saving changes could shift the recorded times; and because participation is voluntary, the data may not represent all players equally. The data were gathered in two parts: a sign-up form (player info and newsletter opt-in) and server logs (session start/end times), joined by the hashed email. We summarize the full dataset here even if we later use only a subset of variables for modeling.



**Vaiable Summary**

**players.csv**  
- `experience` · categorical 
- `subscribe`  · boolean     
- `hashedEmail`· string(key)
- `played_hours`· numeric    
- `name`       · string     
- `gender`     · categorical
- `Age`        · numeric    

**sessions.csv**  
- `hashedEmail` · string(fk) ·  players  
- `start_time` / `end_time` · string · `dd/mm/yyyy HH:MM`   
- `original_start_time` / `original_end_time` · numeric · epoch

2) Questions

Broad question:
Which player characteristics and early behaviours are most predictive of subscribing to the game-related newsletter?

Specific question:
Can basic play behaviour statistics (e.g., total sessions, average session length) and demographic attributes predict whether a player subscribes to the newsletter?